This notebook contains two methods to generate a video by starting with a text input to read using TTS.

The first method adds subtitles to the video that display 1+ line at a time. The second method adds word-synced captions to the video using a modified `autocaption` module.

In [ ]:
# notes on where im trying to go with this
# can provide:
# url which will be extracted to get input text (optionally formatted using LLM)
# file or text which will be used as input text
# video url which will skip the text/TTS and adds subtitles to the video

# so the process involves:

# with TTS audio, generate

In [ ]:
import os, sys

sys.path.append(os.path.abspath('..'))

from src.utils import ensuredir, displayVideo, get_text_from_url
from src.enums import ElevenLabsTTSModel, ElevenLabsTTSVoice
from src.tools.tts import make_tts
from src.tools.stt import create_transcript
from src.video.moviepy import create_video_with_subtitles, create_video_from_audio

# change the model and/or voice if desired
# TODO support openai tts
tts_model = ElevenLabsTTSModel.Multilingual_v2.value
tts_voice = ElevenLabsTTSVoice.Brian.value  # TODO add more voices to enum

# choose 1 of the below inputs
input_text = ""
input_file = ''
input_url = 'http://example.com'
reformat_url_text = False
input_video = '' # TODO

output_file = 'test2.mp3'

method = 'subtitle' # 'subtitle' or 'autocaption'

In [ ]:
from typing import Literal

input_type: Literal['text', 'file', 'url'] = 'text'
input_src = input_text
if input_file:
	input_type = 'file'
	input_src = input_file
elif input_url:
	input_type = 'url'
	input_src = input_url

if input_url:
	res = get_text_from_url(input_url)
	print(res)

In [ ]:
if method == 'subtitle':
	audio_path = make_tts(input_src, input_type, output_file, model=tts_model, voice=tts_voice)

	# Generate SRT file
	transcript = create_transcript(audio_path)

	video_output_file = f"{os.path.splitext(audio_path)[0]}.mp4"
	# save in output/video
	video_output_file = video_output_file.replace('tts', 'video')
	ensuredir(video_output_file)
	create_video_with_subtitles(audio_path, transcript, video_output_file, False)
	print(f'Video saved to {video_output_file}')
	displayVideo(path=video_output_file)

In [ ]:
if method == 'autocaption':
	from autocaption.predict import VideoCaptioner
	import shutil

	# generate tts + video with audio
	audio_path = make_tts(input_src, input_type, output_file, model=tts_model, voice=tts_voice)
	video_output_file = f"{os.path.splitext(audio_path)[0]}.mp4"
	filename = os.path.basename(video_output_file)
	create_video_from_audio(audio_path, video_output_file)

	vc = VideoCaptioner()
	output_paths = vc.add_captions(filename, kerning=-2)
	o = output_paths[0]
	shutil.move(o, video_output_file)
	displayVideo(path=video_output_file)

# for this, still generate the tts
# create video with audio and no subtitles
# pass tts video to vc
# add audio back to output video

In [ ]:
# TODO
# youtube video as input
# add captions to video

# video_input = 'test2.mp4'
# vc = VideoCaptioner()
# outputs = vc.add_captions(video_input, kerning=-2)
# o = outputs[0]
# shutil.move(o, 'test3.mp4')

In [ ]:
# TODO
# - save transcript as word-timestamp json instead of srt, construct and write to srt to control split points
# - display paragraph(s) of text at a time on screen,
#   highlight segments as they're spoken
# - generate background image for the video

# idea for an autonomous version
# a script format that defines the video
# - list info at the beginning (audio, background color, etc)
# - each line after contains a timestamp and and action of some sort
#   - display text
#   - display image
# an llm could output this script and the app generates the video from it